In [2]:
%load_ext ipython_unittest

# Bin Packing Problem with Conflicts

### Algorithm 1: ILS para o PBPC (KLB, ItBL) 

```
S ← ConstroiSoluçãoInicial();
K ← Número de bins em S;
enquanto K ≥ KLB and S é viável, faça: 
  ReduzNúmeroBins(S);
  K ← K − 1;
  ItPERT ← 0;
  enquanto ItPERT ≤ ItMAX and S não é viável, faça: 
    S ← BuscaLocal(S);
    S ← EjectionChains(S);
    se ItBL sucessivas BL e EC sem melhora entao˜
      S ← Perturbação(S);
      ItPERT ← ItPERT + 1;
      ItBL ← 0;
      fim se
    fim enquanto
fim enquanto
```

## Utilidades

In [3]:
def hasConflict(conflicts, item, bins):
    for item_bin in bins["itens"]:
        if item_bin["color"] in conflicts[str(item["color"])]:
            return True
    return False

In [6]:
%%unittest_testcase
conflicts = {
            "0":[2],
            "1":[],
            "2":[0]
        }

bins = {"itens":[{"value":3, "color": 0}], "space_left": 0}

def test_bins_conflicting(self):
    item = {"value":2, "color": 2}
    conflicted = hasConflict(conflicts, item, bins)
    assert conflicted == True

def test_bins_conflicting(self):
    item = {"value":2, "color": 1}
    conflicted = hasConflict(conflicts, item, bins)
    assert conflicted == False

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [18]:
def isSolutionViable(solution, BIN_TOTAL_SPACE, conflicts):
    for bins in solution:
        if bins["space_left"] < 0:
            return False
        for item in bins["itens"]:
            if hasConflict(conflicts, item, bins):
                return False
    return True

In [19]:
%%unittest_testcase

BIN_TOTAL_SPACE = 5
    

fixture = { "bins":[
                {"itens":[{"value":5, "color": 0}], "space_left": 0},
                {"itens":[{"value":4, "color": 0}, {"value":1, "color": 2}], "space_left": 0},
                {"itens":[{"value":3, "color": 0},{"value":2, "color": 1}], "space_left": 0},
            ],
            "conflicts":{
                "0":[],
                "1":[2],
                "2":[1]
            }
          }

def test_solution_viability_with_conflicts(self):
    conflicts = {
        "0":[2],
        "1":[],
        "2":[0]
    }
    
    response = isSolutionViable(fixture["bins"], BIN_TOTAL_SPACE, conflicts)
    assert response == False

def test_solution_viability_with_bins_overloaded(self):
    bins = [
             {"itens":[{"value":5, "color": 0}], "space_left": 0},
             {"itens":[{"value":4, "color": 0}, {"value":1, "color": 2}], "space_left": 0},
             {"itens":[{"value":5, "color": 0},{"value":2, "color": 1}], "space_left": -2},
           ]

    response = isSolutionViable(bins, BIN_TOTAL_SPACE, fixture["conflicts"])
    assert response == False

def test_solution_viable(self):
    response = isSolutionViable(fixture["bins"], BIN_TOTAL_SPACE, fixture["conflicts"])
    assert response == True

Success

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## Algoritmo de construção

In [7]:
def first_fit_decreasing(itens, BIN_TOTAL_SPACE):
    sortedItens = sorted(itens, key=lambda k: k["value"], reverse=True)
    solution = [] #array of bins
    for item in sortedItens:
        if len(solution) == 0:
            solution.append({"itens":[item], "space_left":BIN_TOTAL_SPACE-item["value"]})
            continue
        inserted = False
        for bins in solution:
            if bins["space_left"] >= item["value"] and hasConflict(item, bins) == False:
                bins["itens"].append(item)
                bins["space_left"] -= item["value"]
                inserted = True
                break
                
        if not inserted:
            solution.append({"itens":[item], "space_left":BIN_TOTAL_SPACE-item["value"]})
    return solution

In [9]:
%%unittest_testcase
def test_first_fit(self):
    BIN_TOTAL_SPACE = 5
    fixture = [
                {"value":5},
                {"value":4},
                {"value":3},
                {"value":1},
                {"value":2}
              ]
    
    solutionExpected = [
                        {"itens":[{"value":5}], "space_left": 0},
                        {"itens":[{"value":4},{"value":1}], "space_left": 0},
                        {"itens":[{"value":3},{"value":2}], "space_left": 0},
                     ]
    
    S = first_fit_decreasing(fixture, BIN_TOTAL_SPACE)
    assert S == solutionExpected
    

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>